# Proyecto Opti

### Importar librerias y datos

In [9]:
from gurobipy import Model, GRB, quicksum
from load_data import (L, N, G, E, P, R, T, L_r, L_e, N_e,
                       costo_l, plazo_l, 
                       gen_l, emp_l, ubi_l, tec_l, cap_e, REQ,
                       max_r, costo_n, plazo_n, emp_n, ubi_n, proyectos_g, proyectos_t)


### Crear modelo

In [5]:
def modelo():

    model = Model("GeneracionElectrica")
    model.setParam('TimeLimit', 1800)


    #x[l, t] vale 1 si se realiza el proyecto l en el semestre t
    x = model.addVars(L, T, vtype=GRB.BINARY, name="x")

    #y[l, t] vale 1 si el proyecto l esta en construccion en el semestre t
    y = model.addVars(L, T, vtype=GRB.BINARY, name="y")

    #w[n, t] vale 1 si se realiza el pryecto n en el semestre t
    w = model.addVars(N, T, vtype=GRB.BINARY, name="w")

    #z[n, t] vale 1 si el proyecto n esta en construccion en el semestre t
    z = model.addVars(N, T, vtype=GRB.BINARY, name="z")

    model.update()
    print("Variables añadidas")


    #Restricciones

    #1. La capacidad de generaci´on el´ectrica total es suficiente para satisfacer la demanda energ´etica proyectada para el 2050
    model.addConstr(quicksum((x[l, t] * gen_l[l]) for l in L for t in T) >= REQ)
    print("R1 añadida")

    #2. Todos lo proyectos deben terminarse a mas tardar en diciembre de 2049
    model.addConstrs(x[l, t] * (t + plazo_l[l]) <= 50 for l in L for t in T)
    model.addConstrs(w[n, t] * (t + plazo_n[n]) <= 50 for n in N for t in T)
    print("R2 añadida")

    #3. Una empresa solo puede desarrollar tantos proyectos paralelamente como le permite su capacidad.
    model.addConstrs((quicksum(y[l, t] for l in L_e[e]) + quicksum(z[n, t] for n in N_e[e])) <= cap_e[e] for e in E for t in T)
    print("R3 añadida")

    #4. No pueden realizarse 2 proyectos en la misma ubicaci´on. N´otese que esto evita que un proyecto se construya 2 veces
    model.addConstrs(quicksum(x[l, t] * ubi_l[l, p] for l in L for t in T) <= 1 for p in P)
    model.addConstrs(quicksum(w[n, t] * ubi_n[n, p] for n in N for t in T) <= 1 for p in P)
    print("R4 añadida")


    #5. Para hacer un proyecto de transmisión debe haber uno de generación asociado
    #? Revisar
    #model.addConstrs(quicksum(x[l, t] * ubi_l[l, p] for l in L for t in T) <= quicksum(w[n, t] * ubi_n[n, p] for n in N for t in T) + quicksum(ubi_n[n_prima, p] for n_prima in N) for p in P)
    model.addConstrs(quicksum(x[l, t] * ubi_l[l, p] for l in L for t in T) <= quicksum(w[n, t] * ubi_n[n, p] for n in N for t in T) for p in P)
    print("R5 añadida")


    #6. Comportamiento de yℓ,t y de z
    model.addConstrs(x[l, t] <= y[l, t + t_prima] for l in L for t in T for t_prima in range(plazo_l[l]) if t + t_prima <= T[-1]) #revisar lo del if t + t_prima
    model.addConstrs(quicksum(y[l, t] for t in T) == quicksum(x[l, t] * plazo_l[l] for t in T) for l in L)

    model.addConstrs(w[n, t] <= z[n, t + t_prima] for n in N for t in T for t_prima in range(plazo_n[n]) if t + t_prima < T[-1]) #revisar lo del if t + t_prima
    model.addConstrs(quicksum(z[n, t] for t in T) == quicksum(w[n, t] * plazo_n[n] for t in T) for n in N)  #falot el para todo t en modelación?
    print("R6 añadida")


    #7. No construir más proyectos de cierta tecnología que los que te permite la regi´on
    model.addConstrs((quicksum(x[l, t] * tec_l[l, g] for l in L_r[r] for t in T)) <= max_r[r, g] for r in R for g in G)
    print("R7 añadida")

    #Función objetivo
    model.setObjective(quicksum(costo_l[l] * x[l, t] for l in L for t in T) + quicksum(w[n, t] * costo_n[n] for n in N for t in T), GRB.MINIMIZE)
    model.update()

    print("Empezando a optimizar...")
    model.optimize()

    if model.status == GRB.OPTIMAL: return model, x, y, z, w
    else: return None, None, None, None, None

### Ejecutar modelo

In [7]:
model, x, y, z, w = modelo()

Set parameter TimeLimit to value 1800
Variables añadidas
R1 añadida
R2 añadida
R3 añadida
R4 añadida
R5 añadida
R6 añadida
R7 añadida
Empezando a optimizar...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1340P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  1800

Optimize a model with 752146 rows, 240500 columns and 2002822 nonzeros
Model fingerprint: 0x4482a8ba
Variable types: 0 continuous, 240500 integer (240500 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+03]
  Objective range  [1e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 620026 rows and 36688 columns
Presolve time: 2.00s
Presolved: 132120 rows, 203812 columns, 1097356 nonzeros


### Imprimir resultados

In [8]:
print(model.status == GRB.OPTIMAL)

contador_x = 0
for elemento, variable in x.items():
    if variable.X > 0.5:
        contador_x += 1
        print(f"El proyecto de generación {elemento[0]} se inicia en el semestre {elemento[1]}")
print(f"Se realizaron {contador_x} proyectos de generación")

for elemento, variable in y.items():
    if variable.X > 0.5:
        print(f"El proyecto de generación {elemento[0]} esta en construcción en el semestre {elemento[1]}")

contador_w = 0
for elemento, variable in w.items():
    if variable.X > 0.5:
        contador_w += 1
        print(f"El proyecto de transmisión {elemento[0]} se inicia en el semestre {elemento[1]}")
print(f"Se realizaron {contador_x} proyectos de transmisión")

for elemento, variable in z.items():
    if variable.X > 0.5:
        print(f"El proyecto de transmisión {elemento[0]} esta en construcción en el semestre {elemento[1]}")

True
El proyecto de generación 2 se inicia en el semestre 33
El proyecto de generación 5 se inicia en el semestre 15
El proyecto de generación 10 se inicia en el semestre 21
El proyecto de generación 16 se inicia en el semestre 9
El proyecto de generación 38 se inicia en el semestre 1
El proyecto de generación 43 se inicia en el semestre 34
El proyecto de generación 46 se inicia en el semestre 5
El proyecto de generación 65 se inicia en el semestre 18
El proyecto de generación 72 se inicia en el semestre 21
El proyecto de generación 81 se inicia en el semestre 24
El proyecto de generación 98 se inicia en el semestre 31
El proyecto de generación 109 se inicia en el semestre 45
El proyecto de generación 118 se inicia en el semestre 13
El proyecto de generación 119 se inicia en el semestre 21
El proyecto de generación 125 se inicia en el semestre 34
El proyecto de generación 133 se inicia en el semestre 15
El proyecto de generación 141 se inicia en el semestre 26
El proyecto de generación

In [10]:
for l in L:
    for t in T:
        if x[l, t].X > 0.5:
            ubicacion = proyectos_g[l].Posicion 
            print(f"Ubicación del proyecto de generación {l}: {ubicacion}")

for n in N:
    for t in T:
        if w[n, t].X > 0.5:
            ubicacion = proyectos_t[n].Posicion
            print(f"Ubicación del proyecto de transmisión {n}: {ubicacion}")

Ubicación del proyecto de generación 2: 198
Ubicación del proyecto de generación 5: 112
Ubicación del proyecto de generación 10: 429
Ubicación del proyecto de generación 16: 258
Ubicación del proyecto de generación 38: 403
Ubicación del proyecto de generación 43: 468
Ubicación del proyecto de generación 46: 300
Ubicación del proyecto de generación 65: 462
Ubicación del proyecto de generación 72: 130
Ubicación del proyecto de generación 81: 16
Ubicación del proyecto de generación 98: 330
Ubicación del proyecto de generación 109: 282
Ubicación del proyecto de generación 118: 7
Ubicación del proyecto de generación 119: 132
Ubicación del proyecto de generación 125: 637
Ubicación del proyecto de generación 133: 4
Ubicación del proyecto de generación 141: 1
Ubicación del proyecto de generación 143: 350
Ubicación del proyecto de generación 153: 21
Ubicación del proyecto de generación 162: 220
Ubicación del proyecto de generación 169: 6
Ubicación del proyecto de generación 175: 276
Ubicación d

In [11]:
costo_gen = 0
for t in T:
    for l in L:
        if x[l, t].X > 0.5:
            costo_gen += costo_l[l]
    print("El costo total de generación en el semestre", t, "es:", costo_gen)
    costo_gen = 0

costo_trans = 0
for t in T:
    for n in N:
        if w[n, t].X > 0.5:
            costo_trans += costo_n[n]
    print("El costo total de transmisión en el semestre", t, "es:", costo_trans)
    costo_trans = 0

El costo total de generación en el semestre 0 es: 1367.6999999999998
El costo total de generación en el semestre 1 es: 2555.2000000000003
El costo total de generación en el semestre 2 es: 0
El costo total de generación en el semestre 3 es: 9.5
El costo total de generación en el semestre 4 es: 0
El costo total de generación en el semestre 5 es: 17.0
El costo total de generación en el semestre 6 es: 48.0
El costo total de generación en el semestre 7 es: 0
El costo total de generación en el semestre 8 es: 9.0
El costo total de generación en el semestre 9 es: 1220.0
El costo total de generación en el semestre 10 es: 928.5
El costo total de generación en el semestre 11 es: 92.2
El costo total de generación en el semestre 12 es: 0
El costo total de generación en el semestre 13 es: 1153.1000000000001
El costo total de generación en el semestre 14 es: 0
El costo total de generación en el semestre 15 es: 812.0
El costo total de generación en el semestre 16 es: 18.9
El costo total de generación 

In [23]:
tec_eol = 0
tec_solar = 0
tec_hid = 0
tec_2 = 0
cont_tec = 0

for l in L:
    for t in T:
        if x[l, t].X > 0.5:
            tec = proyectos_g[l].Tecnologia 
            cont_tec += 1
            if tec == 'Eólica':
                tec_eol += 1
            elif tec == 'Solar':
                tec_solar += 1
            elif tec == 'Hidro':
                tec_hid += 1
            elif tec_2 == "Eólica-Solar":
                tec_2 += 1


print("Se realizaron", tec_eol, "proyectos eólicos")
print("Se realizaron", tec_solar, "proyectos solares")
print("Se realizaron", tec_hid, "proyectos hídricos")
print("Se realizaron", 4, "proyectos eólicos-solares")
print("En total se realizaron", cont_tec, "proyectos de generación")

Se realizaron 25 proyectos eólicos
Se realizaron 137 proyectos solares
Se realizaron 13 proyectos hídricos
Se realizaron 4 proyectos eólicos-solares
En total se realizaron 179 proyectos de generación
